## A driver can only take one order at a time

In [1]:
import pickle

import numpy as np
import pandas as pd

from functools import partial
from geopy.distance import geodesic

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
path = "input_data"

READ INPUT DATA FILES

In [3]:
order_df = pd.read_csv(f"{path}/order_data.csv")
order_df.head()

,order_id,restaurant_lat,restaurant_long,customer_lat,customer_long,no_of_items,prep_duration_sec,preferred_otd_sec
0,0,52.534717,13.392920,52.536950,13.363647,9,960,2400
1,1,52.507490,13.460169,52.521861,13.464337,10,780,2400
2,2,52.539029,13.384884,52.507035,13.459327,12,360,2400
3,3,52.508368,13.408875,52.519709,13.374291,16,1020,2400
4,4,52.537509,13.429478,52.544824,13.432814,15,360,2400


In [4]:
driver_df = pd.read_csv(f"{path}/driver_data.csv")
driver_df.head()

,driver_id,shift_start_sec,shift_end_sec,start_location_lat,start_location_long,vehicle
0,0,0,7200,52.502670,13.447406,BIKE
1,1,0,7200,52.518258,13.447747,BIKE
2,2,0,7200,52.517876,13.460788,BIKE
3,3,0,7200,52.528275,13.431436,BIKE
4,4,0,7200,52.510854,13.454662,BIKE


In [5]:
# Check initial sanity of data
order_df.describe()

,order_id,restaurant_lat,restaurant_long,customer_lat,customer_long,no_of_items,prep_duration_sec,preferred_otd_sec
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0
mean,49.500000,52.520640,13.408850,52.518894,13.406724,9.840000,723.600000,2400.0
std,29.011492,0.017904,0.028543,0.019089,0.028775,5.052692,267.489337,0.0
min,0.000000,52.485165,13.353377,52.486500,13.354016,1.000000,300.000000,2400.0
25%,24.750000,52.507638,13.388908,52.502885,13.384448,6.000000,480.000000,2400.0
50%,49.500000,52.519565,13.407955,52.518842,13.401845,9.000000,720.000000,2400.0
75%,74.250000,52.534732,13.429163,52.534923,13.430591,14.250000,960.000000,2400.0
max,99.000000,52.554721,13.464321,52.555948,13.464337,19.000000,1200.000000,2400.0


In [6]:
driver_df.describe()

,driver_id,shift_start_sec,shift_end_sec,start_location_lat,start_location_long
count,40.000000,40.0,40.0,40.000000,40.000000
mean,19.500000,0.0,7200.0,52.520005,13.406796
std,11.690452,0.0,0.0,0.015724,0.030908
min,0.000000,0.0,7200.0,52.488881,13.351823
25%,9.750000,0.0,7200.0,52.509868,13.381836
50%,19.500000,0.0,7200.0,52.519269,13.410317
75%,29.250000,0.0,7200.0,52.530530,13.422828
max,39.000000,0.0,7200.0,52.555571,13.460788


In [7]:
# We have used geodesics to calculate distances between two coordinates
def calculate_distance(coord1, coord2):
    return int(geodesic(coord1, coord2).meters)

In [8]:
pickup_coordinates = order_df[["restaurant_lat", "restaurant_long"]].values.tolist()
drop_coordinates = order_df[["customer_lat", "customer_long"]].values.tolist()

In [9]:
veh_start_coordinates = driver_df[
    ["start_location_lat", "start_location_long"]
].values.tolist()

In [10]:
cust_coordinates = pickup_coordinates + drop_coordinates

In [11]:
len(pickup_coordinates)

100

In [12]:
len(cust_coordinates)

200

In [13]:
all_coordinates = cust_coordinates + veh_start_coordinates
all_coordinates_count = len(all_coordinates)
all_coordinates_count

240

In [14]:
# 1 dummy node at beginning to allow arbitrary end locations of routes
distance_matrix = [
    [0] * (all_coordinates_count + 1) for _ in range(all_coordinates_count + 1)
]

In [15]:
# for i in range(all_coordinates_count):
#     for j in range(all_coordinates_count):
#         distance_matrix[i+1][j+1] = calculate_distance(all_coordinates[i], all_coordinates[j])

# with open("dm", "wb") as fp:   #Pickling
#   pickle.dump(distance_matrix, fp)

In [16]:
with open("dm", "rb") as fp:  # Unpickling
    distance_matrix = pickle.load(fp)

In [17]:
data = {}
data["distance_matrix"] = distance_matrix
data["num_vehicles"] = driver_df.shape[0]

In [18]:
data["veh_start_indices"] = [
    len(cust_coordinates) + i + 1 for i in range(driver_df.shape[0])
]

In [19]:
data["veh_end_indices"] = [0 for _ in range(driver_df.shape[0])]

In [20]:
data["num_vehicles"]

40

In [21]:
len(data["distance_matrix"])

241

In [22]:
driver_df["cost_per_meter"] = driver_df.vehicle.apply(lambda x: 1 if x == "BIKE" else 3)
driver_df["speed_meter_per_sec"] = driver_df.vehicle.apply(
    lambda x: 15 * 1000 / 3600 if x == "BIKE" else 30 * 1000 / 3600
)
driver_df["capacity"] = driver_df.vehicle.apply(lambda x: 10 if x == "BIKE" else 30)

In [23]:
dummy_pickups_drop_demands = (
    [0]
    + order_df["no_of_items"].values.tolist()
    + list(-1 * order_df["no_of_items"].values)
)

In [24]:
veh_start_node_demands = [
    0 for _ in range((all_coordinates_count + 1) - len(dummy_pickups_drop_demands))
]

In [25]:
data["demands"] = dummy_pickups_drop_demands + veh_start_node_demands

In [26]:
data["vehicle_capacities"] = driver_df["capacity"].values.tolist()

In [27]:
data["pickups_deliveries"] = [[i, i + 100] for i in range(1, order_df.shape[0] + 1)]

In [28]:
dummy_location_tw = [[0, 7200]]
pickup_drop_tw = (
    order_df[["prep_duration_sec", "preferred_otd_sec"]].values.tolist() * 2
)
dummy_pickup_drop_tw = dummy_location_tw + pickup_drop_tw

In [29]:
driver_df[["shift_start_sec", "shift_end_sec"]].values.tolist()

[[0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200],
 [0, 7200]]

In [30]:
veh_start_node_tw = [
    [0, 7200]
    for _ in range(all_coordinates_count + 1 - len(dummy_pickups_drop_demands))
]

In [31]:
data["time_windows"] = dummy_pickup_drop_tw + veh_start_node_tw

In [32]:
# Create the Routing Index Manager
manager = pywrapcp.RoutingIndexManager(
    len(data["distance_matrix"]),
    data["num_vehicles"],
    data["veh_start_indices"],
    data["veh_end_indices"],
)

In [33]:
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

Arc Costs being defined : START

In [34]:
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)

    # Calculate distance between nodes
    distance = data["distance_matrix"][from_node][to_node]

    return distance


def cost_callback(from_index, to_index, cost_per_meter):
    distance = distance_callback(from_index, to_index)
    cost = distance * cost_per_meter

    return cost


def travel_time_callback(from_index, to_index, speed_in_mps):
    distance = distance_callback(from_index, to_index)
    time = distance / speed_in_mps

    return int(round(time, 0))

In [35]:
bike_transit_cost_callback = partial(cost_callback, cost_per_meter=1)
car_transit_cost_callback = partial(cost_callback, cost_per_meter=3)

In [36]:
distance_callback_index = routing.RegisterTransitCallback(distance_callback)

bike_transit_cost_callback_index = routing.RegisterTransitCallback(
    bike_transit_cost_callback
)
car_transit_cost_callback_index = routing.RegisterTransitCallback(
    car_transit_cost_callback
)

In [37]:
# Define cost of each arc.
for _, row in driver_df.iterrows():
    v_id = row["driver_id"]
    v_type = row["vehicle"]
    if v_type == "BIKE":
        routing.SetArcCostEvaluatorOfVehicle(bike_transit_cost_callback_index, v_id)
    elif v_type == "CAR":
        routing.SetArcCostEvaluatorOfVehicle(car_transit_cost_callback_index, v_id)

In [38]:
# Add Distance dimension to track distances
dimension_name = "Distance"
routing.AddDimension(
    distance_callback_index,
    0,  # no slack
    int(sum(data["distance_matrix"][1]))
    * 1000,  # vehicle maximum travel distance to be large
    True,  # start cumul to zero
    dimension_name,
)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
# distance_dimension.SetGlobalSpanCostCoefficient(100)

Arc Costs being defined : END

In [39]:
# [START pickup_delivery_constraint]
for request in data["pickups_deliveries"]:
    pickup_index = manager.NodeToIndex(request[0])
    delivery_index = manager.NodeToIndex(request[1])
    routing.AddPickupAndDelivery(pickup_index, delivery_index)
    routing.solver().Add(
        routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
    )
    routing.solver().Add(
        distance_dimension.CumulVar(pickup_index)
        <= distance_dimension.CumulVar(delivery_index)
    )
# [END pickup_delivery_constraint]

### ADDING CAPACITY CONSTRAINTS

In [40]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data["demands"][from_node]

In [41]:
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data["vehicle_capacities"],  # vehicle maximum capacities
    True,  # start cumul to zero
    "Capacity",
)

True

### ADDING TIME WINDOW CONSTRAINTS

In [42]:
bike_travel_time_callback = partial(travel_time_callback, speed_in_mps=15 * 5 / 18)
car_travel_time_callback = partial(travel_time_callback, speed_in_mps=30 * 5 / 18)

bike_travel_time_callback_index = routing.RegisterTransitCallback(
    bike_travel_time_callback
)
car_travel_time_callback_index = routing.RegisterTransitCallback(
    car_travel_time_callback
)

In [43]:
travel_time_callback_indices = driver_df.vehicle.apply(
    lambda x: bike_travel_time_callback_index
    if x == "BIKE"
    else car_travel_time_callback_index
).to_list()

In [44]:
dimension_name = "Time"
routing.AddDimensionWithVehicleTransitAndCapacity(
    travel_time_callback_indices,
    7200,  # Max Slack
    [7200] * driver_df.shape[0],  # vehicle maximum travel time
    False,  # don't start cumul to zero
    dimension_name,
)

time_dimension = routing.GetDimensionOrDie(dimension_name)

In [45]:
penalty = 1  # $1/second
for location_idx, time_window in enumerate(data["time_windows"]):
    if location_idx in data["veh_start_indices"] + data["veh_end_indices"]:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetMin(time_window[0])
    time_dimension.SetCumulVarSoftUpperBound(index, time_window[1], penalty)

In [46]:
for vehicle_id in range(data["num_vehicles"]):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(0, 7200)

In [47]:
# Instantiate route start and end times to produce feasible times.
# [START depot_start_end_times]
for i in range(data["num_vehicles"]):
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))
# [END depot_start_end_times]

In [48]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC
)

# search_parameters.time_limit.seconds = 30

In [49]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [50]:
print(f"Objective Cost: ${solution.ObjectiveValue()}")
time_dimension = routing.GetDimensionOrDie("Time")
distance_dimension = routing.GetDimensionOrDie("Distance")
total_time = 0
total_distance = 0
max_route_cost = 0
total_route_cost = 0
total_delay = 0
for vehicle_id in range(data["num_vehicles"]):
    index = routing.Start(vehicle_id)
    plan_output = f"Route for vehicle {vehicle_id}:\n"
    route_cost = 0
    route_distance = 0
    route_penalty_cost = 0
    route_delay = 0
    while not routing.IsEnd(index):
        time_var = time_dimension.CumulVar(index)
        dist_var = distance_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
            " -> "
        )
        if solution.Min(time_var) >= 2400:
            route_penalty_cost += (solution.Min(time_var) - 2400) * 1
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += solution.Value(dist_var)
        route_cost += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

    time_var = time_dimension.CumulVar(index)
    dist_var = distance_dimension.CumulVar(index)

    plan_output += (
        f"{manager.IndexToNode(index)}"
        f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
    )

    route_delay += route_penalty_cost
    route_cost += route_penalty_cost * penalty
    plan_output += f"Time of the route: {solution.Min(time_var)} seconds\n"
    plan_output += f"Cost of the route: ${route_cost}\n"
    plan_output += f"Distance of the route: {solution.Min(dist_var)}\n"
    plan_output += f"Delay of the route: {route_delay}"
    print(plan_output)
    max_route_cost = max(route_cost, max_route_cost)
    total_route_cost += route_cost
    total_time += solution.Min(time_var)
    total_distance += route_distance
    total_delay += route_delay
print(f"Maximum of the route costs: ${max_route_cost}")
print(f"Total route cost: ${total_route_cost}")
print(f"Total route time: {total_time} seconds")
print(f"Total route distance: {total_distance} meters")
print(f"Total route delay: {total_delay} seconds")

Objective Cost: $636478
Route for vehicle 0:
201 Time(0,0) -> 89 Time(1080,1080) -> 27 Time(1308,1308) -> 127 Time(2146,2146) -> 17 Time(2368,2368) -> 189 Time(2405,2405) -> 117 Time(3513,3513) -> 0 Time(3513,3513)
Time of the route: 3513 seconds
Cost of the route: $11810
Distance of the route: 10692
Delay of the route: 1118
Route for vehicle 1:
202 Time(0,0) -> 14 Time(360,360) -> 21 Time(827,827) -> 80 Time(923,923) -> 121 Time(1287,1287) -> 114 Time(1815,1815) -> 180 Time(2316,2316) -> 0 Time(2316,2316)
Time of the route: 2316 seconds
Cost of the route: $8381
Distance of the route: 8381
Delay of the route: 0
Route for vehicle 2:
203 Time(0,0) -> 67 Time(420,1074) -> 59 Time(1200,1200) -> 167 Time(2079,2079) -> 47 Time(2316,2316) -> 159 Time(3020,3020) -> 147 Time(3585,3585) -> 0 Time(3585,3585)
Time of the route: 3585 seconds
Cost of the route: $13247
Distance of the route: 11442
Delay of the route: 1805
Route for vehicle 3:
204 Time(0,0) -> 43 Time(420,420) -> 46 Time(833,833) -> 1

In [51]:
(60374, 7788), (67038, 18321), (96719, 96719), (612644, 79669), (620521, 71814), (
    623196,
    55842,
)

((60374, 7788),
 (67038, 18321),
 (96719, 96719),
 (612644, 79669),
 (620521, 71814),
 (623196, 55842))